In [ ]:
#https://huggingface.co/blog/davidberenstein1957/fine-tune-modernbert-on-synthetic-data

In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers==4.48.0
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 78.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import datasets

from datasets import load_dataset
from datasets.arrow_dataset import Dataset
from datasets.dataset_dict import DatasetDict, IterableDatasetDict
from datasets.iterable_dataset import IterableDataset
import transformers
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import evaluate
import pandas as pd
import numpy as np



In [ ]:
text1 = "Natalie Morris:  Yo"  # withName example
text2 = "Game history: Natalie Morris:  YoDetect deception for the following message: Natalie Morris:  Yo"  # msgAndHistory example

tokens1 = tokenizer(text1, return_tensors="pt")
tokens2 = tokenizer(text2, return_tensors="pt")

print("WithName tokens:", tokens1.input_ids.shape)
print("MsgAndHistory tokens:", tokens2.input_ids.shape)

WithName tokens: torch.Size([1, 9])
MsgAndHistory tokens: torch.Size([1, 22])


In [8]:
import sklearn
from sklearn.model_selection import train_test_split
dataFile = '/content/drive/MyDrive/combopreppedMafia.tsv'
df=pd.read_csv(dataFile, sep='\t')
print(df.columns)

df = df.drop(["game", "Unnamed: 0.1", "Unnamed: 0", "msgAndHistory", "message", "history","mId", "role","name", "time", 'withName', 'Unnamed: 0.2', 'hiscombo' ], axis=1)
#df['msgAndHistory'] = df.apply(dataFix, axis = 1)
df = df.rename(columns={'label': 'labels','nohiscombo': 'text'})
print(df)

train, validate = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)


mtrain = Dataset.from_pandas(train)
meval = Dataset.from_pandas(validate)

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'mId', 'name', 'role',
       'message', 'label', 'time', 'game', 'withName', 'history',
       'msgAndHistory', 'nohiscombo', 'hiscombo'],
      dtype='object')
     labels                                               text
0         D  {"Task" : "You will be given a message from th...
1         D  {"Task" : "You will be given a message from th...
2         D  {"Task" : "You will be given a message from th...
3         D  {"Task" : "You will be given a message from th...
4         T  {"Task" : "You will be given a message from th...
...     ...                                                ...
2202      T  {"Task" : "You will be given a message from th...
2203      T  {"Task" : "You will be given a message from th...
2204      T  {"Task" : "You will be given a message from th...
2205      D  {"Task" : "You will be given a message from th...
2206      D  {"Task" : "You will be given a message from th...

[2207 rows x 2 columns]


In [9]:
d = df.loc[df["labels"] == "D"]
print(d)

     labels                                               text
0         D  {"Task" : "You will be given a message from th...
1         D  {"Task" : "You will be given a message from th...
2         D  {"Task" : "You will be given a message from th...
3         D  {"Task" : "You will be given a message from th...
7         D  {"Task" : "You will be given a message from th...
...     ...                                                ...
2187      D  {"Task" : "You will be given a message from th...
2192      D  {"Task" : "You will be given a message from th...
2198      D  {"Task" : "You will be given a message from th...
2205      D  {"Task" : "You will be given a message from th...
2206      D  {"Task" : "You will be given a message from th...

[813 rows x 2 columns]


In [10]:

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
#def tokenize(batch):

#    return tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")

# Tokenize dataset
#if "label" in split_dataset["train"].features.keys():
#    split_dataset =  split_dataset.rename_column("label", "labels") # to match Trainer


def preprocess_function(examples):
    label_mapping = {"T": 0, "D": 1}

    res  = tokenizer(examples["text"], padding='max_length', truncation=True, max_length=512)
    res['labels'] = [label_mapping[label] for label in examples['labels']]

    return res

tokenized_train = mtrain.map(preprocess_function, batched=True, remove_columns=["text"])
tokenized_eval = meval.map(preprocess_function, batched=True, remove_columns=["text"])

#tokenized_dataset["train"].features.keys()
# dict_keys(['labels', 'input_ids', 'attention_mask'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/2096 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
print(tokenized_train)

Dataset({
    features: ['labels', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 2096
})


In [ ]:
print(transformers.__version__)

4.48.0


In [12]:

# Model id to load the tokenizer
model_id = "answerdotai/ModernBERT-base"

# Prepare model labels - useful for inference
labels = ["T", "D"]#tokenized_dataset["train"].features["labels"].names


num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
#crashes if not 4.48
# Download the model from huggingface.co/models
model = AutoModelForSequenceClassification.from_pretrained(
    model_id, num_labels=num_labels, label2id=label2id, id2label=id2label,
)


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    pred = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=pred, references=labels)


In [14]:
!pip install 'accelerate>=0.26.0'

In [15]:
from transformers import EarlyStoppingCallback


In [16]:
# Define training args
training_args = TrainingArguments(
    output_dir= "/content/drive/MyDrive/models/mbertMafianhCombofmt",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    num_train_epochs=15,
    bf16=True, # bfloat16 training
    optim="adamw_torch_fused", # improved optimizer
    # logging & evaluation strategies
    logging_strategy="steps",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    #use_mps_device=True,
    metric_for_best_model="accuracy",
    report_to="none",

)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]

)


In [17]:
trainer.train()

W0516 11:47:59.047000 170 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:1948: UserWarning: Tesla T4 does not support bfloat16 compilation natively, skipping
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.712300,0.778111,0.558559
2,0.559300,0.168911,0.954955
3,0.196800,0.094383,0.981982
4,0.060200,0.024343,0.981982
5,0.060800,0.027858,0.990991
6,0.028500,0.000523,1.000000
7,0.028400,0.094558,0.981982
8,0.011800,0.159478,0.963964
9,0.029500,0.000113,1.000000
10,0.007300,0.000076,1.000000


TrainOutput(global_step=1441, training_loss=0.12963428539608518, metrics={'train_runtime': 4139.0733, 'train_samples_per_second': 7.596, 'train_steps_per_second': 0.475, 'total_flos': 7856512897744896.0, 'train_loss': 0.12963428539608518, 'epoch': 11.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.0005233363481238484,
 'eval_accuracy': 1.0,
 'eval_runtime': 6.8943,
 'eval_samples_per_second': 16.1,
 'eval_steps_per_second': 1.015,
 'epoch': 11.0}